In [10]:
import torch
import requests
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer, util

In [90]:
def get_gen_ai_response(prompt1, prompt2):
    model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

    description1 = prompt1
    description2 = prompt2

    embedding1 = model.encode(description1, convert_to_tensor=True)
    embedding2 = model.encode(description2, convert_to_tensor=True)

    similarity = util.cos_sim(embedding1, embedding2).item()

    if similarity > 0.9:
        classification = 'Identical'
    elif similarity > 0.5:
        classification = 'Similar'
    else:
        classification = 'Non-related'

    return similarity, classification

In [91]:
def parse_product(json):
    return json['title'] + ' con precio ' + str(json['base_price']) + json['currency_id'] + ' en condicion ' + json['condition'] + ' del dominio ' + json['domain_id'] + ' con los siguientes atributos: ' + ', '.join(list(map(lambda x: x['name'] + ': ' + x['value_name'], json['attributes'])))

In [92]:
def determine_similarity_items(first_id, second_id):

    first_product = requests.get(f'https://api.mercadolibre.com/items/{first_id}').json()
    second_product = requests.get(f'https://api.mercadolibre.com/items/{second_id}').json()

    first_product_detail = parse_product(first_product)
    second_product_detail = parse_product(second_product)

    print(first_product_detail)
    print(second_product_detail)

    _, classification = get_gen_ai_response(first_product_detail, second_product_detail)

    print(_)

    return classification


In [93]:
determine_similarity_items('MLA1149401082', 'MLA1447421381')

Perfume Masculino Natura Beijo De Humor 75ml Amaderado Moder con precio 29992.5ARS en condicion new del dominio MLA-PERFUMES con los siguientes atributos: Unidades por envase: 1, Número de legajo resolución 155/98: 312437, Formato de aplicación: Spray, Marca: Natura, Género: Hombre, Es recargable: No, Condición del ítem: Nuevo, Línea: Humor, Familia olfativa: Amaderado moderado, Nombre del perfume: Beijo de Humor, Tipo de perfume: Deo colonia, Características del producto: Frágil,Contiene líquido,Con vencimiento, Embalaje del envío: Bolsa, Versión: MASCULINO 75ML
David Beckham Classic Blue Edt 100 Ml con precio 34177ARS en condicion new del dominio MLA-PERFUMES con los siguientes atributos: Número de legajo resolución 155/98: 2350, Formato de aplicación: Spray, Marca: David Beckham, Género: Hombre, Código universal de producto: 3616303461980,03616303461980, Es libre de crueldad: No, Es recargable: No, Es vegano: No, Condición del ítem: Nuevo, Línea: Classic Blue, Familia olfativa: Flor

'Similar'

Se podria mejorar el codigo sustancialmente agregando comparaciones por imagenes con algun modelo de computer vision y los pixeles de las imagenes provistas.

No se utilizo la API de ChatGPT o LLAMA2 por no contar con creditos suficientes. Otros modelos de AI prompt-based no daban resultados aceptables para el challenge y se decidio resolver por este camino alternativo similar.